In [ ]:
import ta
import ccxt
import time
import pprint
import pandas as pd
import SecretConfig
from IPython.display import clear_output

# Définir l'échange
exchange = ccxt.bitget({
    "apiKey":SecretConfig.APIKey,
    "secret":SecretConfig.SecretKey,
    'password': SecretConfig.Passphrase,
    "options": {'defaultType': 'swap', 'adjustForTimeDifference' : True},
    "timeout": 60000,
    "enableRateLimit": True,
})

In [ ]:
# Initialiser un DataFrame vide pour stocker les données roulantes
full_rolling_dataframe = pd.DataFrame()
df_top_10 = pd.DataFrame()

# Définir une fonction pour récupérer et traiter les données
def fetch_data():
    global full_rolling_dataframe
    global df_top_10

    while True:
        # Récupérer les données des contrats à terme
        futures_tickers = exchange.fetch_tickers(params={'type': 'swap'})
        tickers_name_list = futures_tickers.keys()

        # Initialiser un dictionnaire pour stocker les nouvelles données
        data = {}

        # Parcourir chaque contrat à terme
        for ticker in tickers_name_list:

            # Récupérer les informations pertinentes pour chaque contrat à terme
            ticker_name = ticker.split('/')[0]
            data[ticker_name + ' Last Price'] = futures_tickers.get(ticker, {}).get('info', {}).get('last')
            data[ticker_name + ' EMA_1'] = None
            data[ticker_name + ' Var EMA_1'] = None
            data[ticker_name + ' EMA_2'] = None
            data[ticker_name + ' Ecart Carre EMA_1_2'] = None
            data[ticker_name + ' Divergence EMA_1_2'] = None
            data[ticker_name + ' Divergence Moyen Wind1'] = None
            


        # Convertir le dictionnaire en DataFrame
        data['Timestamp'] = futures_tickers.get(ticker, {}).get('timestamp')
        new_data_df = pd.DataFrame([data])

        # Convertir les colonnes en numérique et définir 'Timestamp' comme index
        new_data_df = new_data_df.apply(pd.to_numeric)
        new_data_df['Timestamp'] = pd.to_datetime(new_data_df['Timestamp'], unit='ms')
        new_data_df['Timestamp'] = new_data_df['Timestamp'].dt.round('10L')
        new_data_df = new_data_df.set_index('Timestamp')

        # Ajouter les nouvelles données au DataFrame roulant
        full_rolling_dataframe = pd.concat([full_rolling_dataframe, new_data_df])

        wind1 = 3
        wind2 = 12

        # Calculer les indicateurs techniques
        for ticker in tickers_name_list:
            ticker_name = ticker.split('/')[0]
            full_rolling_dataframe[ticker_name + ' EMA_1'] = ta.trend.ema_indicator(full_rolling_dataframe[ticker_name + ' Last Price'], window= wind1)
            if len(full_rolling_dataframe) >= wind1:
                full_rolling_dataframe[ticker_name + ' Var EMA_1'] = ((full_rolling_dataframe[ticker_name + ' EMA_1'] - full_rolling_dataframe[ticker_name + ' EMA_1'].iloc[- wind1] ) 
                                                                      / full_rolling_dataframe[ticker_name + ' EMA_1'].iloc[- wind1]) * 100
            else:
                full_rolling_dataframe[ticker_name + ' Var EMA_1']  = None
            full_rolling_dataframe[ticker_name + ' EMA_2'] = ta.trend.ema_indicator(full_rolling_dataframe[ticker_name + ' Last Price'], window= wind2)
            full_rolling_dataframe[ticker_name + ' Diff EMA_1_2'] = full_rolling_dataframe[ticker_name + ' EMA_1'] - full_rolling_dataframe[ticker_name + ' EMA_2']
            full_rolling_dataframe[ticker_name + ' Divergence EMA_1_2']= full_rolling_dataframe[ticker_name + ' Diff EMA_1_2'].pct_change(fill_method=None)
            full_rolling_dataframe[ticker_name + ' Divergence Moyen Wind1'] = ta.trend.ema_indicator(full_rolling_dataframe[ticker_name + ' Divergence EMA_1_2'], window= wind1)
            full_rolling_dataframe = full_rolling_dataframe.copy()

        # Convertir en données numériques
        full_rolling_dataframe = full_rolling_dataframe.apply(pd.to_numeric)

        # Limiter le DataFrame roulant aux 600 dernières périodes
        full_rolling_dataframe = full_rolling_dataframe.tail(50)

        # Sélectionnez les colonnes de variation
        cols_variation = [col for col in full_rolling_dataframe.columns if ' Divergence Moyen Wind1' in col]

        # Obtenez la dernière ligne du DataFrame
        last_row = full_rolling_dataframe.iloc[-1]

        # Créez une série avec seulement les colonnes de variation de la dernière ligne
        last_row_variation = last_row[cols_variation]

        # Obtenez les 10 plus grandes variations
        top_10_variations = last_row_variation.nlargest(10)

        # Créez un nouveau DataFrame avec seulement les 10 plus grandes variations
        if df_top_10.empty:
            df_top_10 = pd.DataFrame(top_10_variations)
        else:
            df_top_10 = pd.concat([df_top_10, top_10_variations])

        df_top_10 = df_top_10.tail(10)
        series = df_top_10.stack()
        counts = series.value_counts()


        # Afficher les 10 dernières lignes du DataFrame
        clear_output(wait=True)
        display(full_rolling_dataframe.tail(20))
        display(df_top_10)
        display(counts)
        # print(df_top_10)
        # pprint.pprint(full_rolling_dataframe[ticker_name + ' ATR'])
        # pprint.pprint(full_rolling_dataframe[ticker_name + ' ATR Change'])
        # pprint.pprint(full_rolling_dataframe)

        # Attendre 1 seconde avant de récupérer à nouveau les données
        time.sleep(0.5)



fetch_data()